## Utils

### Imports

In [1]:
import requests
from environments.alfworld.common.environment import ALFWorldEnv
from environments.alfworld.adapt.generate_prompts import generate_openai_tools_prompt
from environments.alfworld.adapt.plan_prompts import plan_prompt
from environments.alfworld.adapt.plan_output_parser import ALFWorldADaPTPPlanOutputParser
import os
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from planning_library.strategies import ADaPTStrategy


%load_ext autoreload
%autoreload 2

### Logging

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ALFWorld + ADaPT"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

## Environment

In [3]:
os.environ["ALFWORLD_DATA"] = "utils/data"

if not "base_config.yaml" in os.listdir("utils"):
    response = requests.get("https://raw.githubusercontent.com/alfworld/alfworld/master/configs/base_config.yaml")
    
    with open("utils/base_config.yaml", "wb") as f:
        f.write(response.content)

env = ALFWorldEnv(config_path="utils/base_config.yaml")

Initializing AlfredTWEnv...


100%|██████████| 8810/8810 [00:03<00:00, 2921.08it/s]

Overall we have 3553 games in split=train
Training with 3553 games


In [4]:
obs, info = env.reset()
print(obs)

-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.


## Strategy

### Hyperparameters

In [5]:
# ADaPT hyperparameters
max_depth = 3  # maximum decomposition depth
executor_max_iterations = 20  # maximum number of iterations for the executor; when reached, the loop will exit

# other hyperparameters
model_name = "gpt-3.5-turbo"
temperature = 0.8

### Executor

Default Executor is powered by any strategy from the library. By default, a simple one is used (see `planning_library/strategies/simple/simple_strategy.py` for more details).

### Prompt

Here is an example of a simple prompt suitable for ALFWorld with ADaPT.

In [6]:
generate_openai_tools_prompt.input_variables

['agent_scratchpad', 'inputs']

In [7]:
print(generate_openai_tools_prompt.format(inputs=obs, agent_scratchpad=[]))

System: You are ALFRED, an intelligent agent navigating in a household.
Human: -= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.
Human: Use observations from an environment to judge whether you have solved the task successfully. When you are sure that you have successfully completed the task, make sure to include the words 'task completed' in your output. Do not write 'task completed' if the task has not been completed.


In [8]:
tools = env.tools
tools

[GoToTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 OpenTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 CloseTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 TakeTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 PutTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 ToggleTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 HeatTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 CoolTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 CleanTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 ExamineTool(env=<textworld.gym.envs.textworld_batch.TextworldBatchGymEnv object at 0x16fba29d0>),
 InventoryTool(env=<textworld.gym.

### Putting It All Together

Let's use [OpenAI Tools](https://python.langchain.com/docs/modules/agents/agent_types/openai_tools) agent.

In [9]:
from langchain.agents import create_openai_tools_agent
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# copied from langchain.agents.create_openai_tools_agent to allow for custom inputs processing
llm = ChatOpenAI(model=model_name, temperature=temperature).bind(tools=[convert_to_openai_tool(tool) for tool in tools])

agent = (
    {
        "inputs": itemgetter("inputs") | RunnableLambda(itemgetter("inputs")),
        "agent_scratchpad": itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_tool_messages),
    }
    | generate_openai_tools_prompt
    | llm
    | OpenAIToolsAgentOutputParser()
)

### Planner

Default Planner is powered by a [`Runnable`](https://python.langchain.com/docs/expression_language/interface) that takes in:

* `inputs`: original inputs
* `intermediate_steps`: a sequence of agent actions
* `agent_outcome`: final answer

It should output a sequence of subtasks and a logic for executing those subtasks (either and or or).

### Prompt

In [10]:
plan_prompt.input_variables

['agent_outcome', 'inputs', 'intermediate_steps']

In [11]:
print(plan_prompt.format(inputs=obs, intermediate_steps=[], agent_outcome="<something>"))

System: You are an advanced reasoning agent that can create plans for reasoning tasks.
Human: 
The task at hand is a household navigation. An attempt to solve it directly has failed, and you need to create a clear and concise plan instead.

You will be given the task description, all the intermediate steps that were taken during the last trial and the final outcome. 

Task description:
-= Welcome to TextWorld, ALFRED! =-

You are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.

Your task is to: put some pencil on shelf.
Human: Intermediate steps:
Human: Final outcome: <something>
Human: 
Please, output a step-by-step plan for how to solve the task successfully. Use the following format:
```
Step 1: <Description of first step>
Step 2: <Description of second step>
...
Step N: <Description of Nth step>
```


### Putting It All Together

In [12]:
planner_runnable = (
    {
        "inputs": itemgetter("inputs") | RunnableLambda(itemgetter("inputs")),
        "agent_outcome": itemgetter("agent_outcome") | RunnableLambda(lambda x: x.log),
        "intermediate_steps": itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_tool_messages),
    }
    | plan_prompt
    | ChatOpenAI(model=model_name, temperature=temperature)
    | ALFWorldADaPTPPlanOutputParser()
).with_config(run_name="Generate Plan")

## Defining strategy

In [13]:
from planning_library.action_executors import GymnasiumActionExecutor


action_executor = GymnasiumActionExecutor(env)
adapt = ADaPTStrategy.create(
    agent=agent,
    tools=tools,
    action_executor=action_executor,
    planner_runnable=planner_runnable,
    executor_max_iterations=executor_max_iterations,
    max_depth=max_depth,
    executor_return_intermediate_steps=True,
    executor_return_finish_log=True,
    return_intermediate_steps=True,
)

In [14]:
adapt.invoke(
    {"inputs": {"inputs": obs}}
)



> Entering new ADaPTStrategy chain...


> Entering new SimpleStrategy chain...
Task completed. The pencil has been successfully put on the shelf.

> Finished chain.
Task completed. The pencil has been successfully put on the shelf.

> Finished chain.


{'inputs': {'inputs': '-= Welcome to TextWorld, ALFRED! =-\n\nYou are in the middle of a room. Looking quickly around you, you see a bed 1, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a desk 1, a drawer 2, a drawer 1, a garbagecan 1, a shelf 1, and a sidetable 1.\n\nYour task is to: put some pencil on shelf.'},
 'intermediate_steps': [[(OpenAIToolAgentAction(tool='goto', tool_input={'receptable_type': 'shelf', 'receptable_id': 1}, log="\nInvoking: `goto` with `{'receptable_type': 'shelf', 'receptable_id': 1}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_wUX8oCgyzqSub034JLHVm2R7', 'function': {'arguments': '{"receptable_type":"shelf","receptable_id":1}', 'name': 'goto'}, 'type': 'function'}]})], tool_call_id='call_wUX8oCgyzqSub034JLHVm2R7'),
    {'observation': 'You arrive at loc 9. On the shelf 1, you see a keychain 2, and a pen 1.',
     'reward': 0,
     'terminated': False,
     'truncated': False,
     'info': {}